In [20]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [18]:
# for column in df_content.columns:
#     # Skip non-numeric columns
#     if df_content[column].dtype.kind in 'biufc':
#         plt.figure(figsize=(10, 6))
#         # Create a histogram and a density plot for the column
#         sns.histplot(df_content[column], kde=True, bins=30)
#         plt.title(f'Distribution of {column}')
#         plt.xlabel(column)
#         plt.ylabel('Frequency')
#         plt.show()

In [19]:
# for column in df_content2.columns:
#     # Skip non-numeric columns
#     if df_content2[column].dtype.kind in 'biufc':
#         plt.figure(figsize=(10, 6))
#         # Create a histogram and a density plot for the column
#         sns.histplot(df_content2[column], kde=True, bins=30)
#         plt.title(f'Distribution of {column}')
#         plt.xlabel(column)
#         plt.ylabel('Frequency')
#         plt.show()

In [21]:
import pandas as pd
import numpy as np

def load_content():
    df_content = pd.read_json('data/content.jsonl', lines=True)

    # getting the rotten tomatoes ratings
    rt_ratings = []
    for ratings_list in df_content['Ratings']:
        rt_rating = next((item['Value'] for item in ratings_list if item['Source'] == 'Rotten Tomatoes'), None)
        if rt_rating:
            rt_rating = int(rt_rating[:-1])
        rt_ratings.append(rt_rating)
    df_content['rtRating'] = rt_ratings

    # getting useful columns
    data_content = df_content[['ItemId', 'Metascore', 'imdbRating', 'imdbVotes', 'rtRating']]

    # replacing string 'N/A' to np.nan and , number separator
    data_content = data_content.replace('N/A', np.nan)
    data_content['imdbVotes'] = data_content['imdbVotes'].str.replace(',', '')

    # converting to numeric data
    data_content['Metascore'] = data_content['Metascore'].astype('float32')
    data_content['imdbVotes'] = data_content['imdbVotes'].astype('float32')

    # Convert imdbRating to numeric, forcing non-numeric values to NaN
    data_content['imdbRating'] = pd.to_numeric(data_content['imdbRating'], errors='coerce')

    # Handle NaN values for imdbRating
    data_content['imdbRating'].fillna(data_content['imdbRating'].median(), inplace=True)

    # Normalizing imdbRating between 0 and 10
    min_rating = data_content['imdbRating'].min()
    max_rating = data_content['imdbRating'].max()
    data_content['normalized_imdbRating'] = 0 + ((data_content['imdbRating'] - min_rating) * (10 - 0)) / (max_rating - min_rating)

    return data_content


In [22]:
df_content = load_content()

In [23]:
df_content

,ItemId,Metascore,imdbRating,imdbVotes,rtRating,normalized_imdbRating
0,c9f0f895fb,NaN,5.5,1980.0,NaN,5.000000
1,d3d9446802,NaN,6.9,6633.0,NaN,6.555556
2,c20ad4d76f,NaN,7.5,11407.0,NaN,7.222222
3,8e296a067a,NaN,4.2,39.0,NaN,3.555556
4,54229abfcf,NaN,6.7,3268.0,NaN,6.333333
...,...,...,...,...,...,...
38007,6c0ffc79d0,NaN,6.2,3202.0,NaN,5.777778
38008,e02f371f8c,NaN,6.3,418.0,NaN,5.888889
38009,8c2a2a22b8,NaN,7.2,1779.0,NaN,6.888889
38010,ae74ba6bb7,NaN,6.5,5.0,NaN,6.111111


In [ ]:
means = df_content.mean(numeric_only=True)
df_content2 = df_content.fillna(means)